# Multiple Linear Regression

Multiple Linear Regression on startup data. Data set contains R&D spend, Administration spend,Marketing spend and State as independent variables, over all profit needs to be predicted based on given independent variables using SK learn and other libraries. This note book also tries to figure out what all features in the dataset are actually important for making a good prediction, i.e Backward elimination is implemented to figure out which independent variables are statistically significant and
which are not significant

In [57]:
#importing libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
print("Numpy version: "+ str(np.__version__))
print("Pandas version: "+ str(pd.__version__))

Numpy version: 1.14.3
Pandas version: 0.23.0


In [44]:
#load data set as data frame
dataset=pd.read_csv('Startups.csv')
print(dataset)

    R&D Spend  Administration  Marketing Spend       State     Profit
0   165349.20       136897.80        471784.10    New York  192261.83
1   162597.70       151377.59        443898.53  California  191792.06
2   153441.51       101145.55        407934.54     Florida  191050.39
3   144372.41       118671.85        383199.62    New York  182901.99
4   142107.34        91391.77        366168.42     Florida  166187.94
5   131876.90        99814.71        362861.36    New York  156991.12
6   134615.46       147198.87        127716.82  California  156122.51
7   130298.13       145530.06        323876.68     Florida  155752.60
8   120542.52       148718.95        311613.29    New York  152211.77
9   123334.88       108679.17        304981.62  California  149759.96
10  101913.08       110594.11        229160.95     Florida  146121.95
11  100671.96        91790.61        249744.55  California  144259.40
12   93863.75       127320.38        249839.44     Florida  141585.52
13   91992.39       

In [45]:
dataset.describe()

,R&D Spend,Administration,Marketing Spend,Profit
count,50.000000,50.000000,50.000000,50.000000
mean,73721.615600,121344.639600,211025.097800,112012.639200
std,45902.256482,28017.802755,122290.310726,40306.180338
min,0.000000,51283.140000,0.000000,14681.400000
25%,39936.370000,103730.875000,129300.132500,90138.902500
50%,73051.080000,122699.795000,212716.240000,107978.190000
75%,101602.800000,144842.180000,299469.085000,139765.977500
max,165349.200000,182645.560000,471784.100000,192261.830000


In [46]:
#generate independent and dependent features 
X=dataset.iloc[:,:-1].values
print("Shape of X is : "+str(X.shape))
y=dataset.iloc[:,4].values
print("Shape of y is : "+str(y.shape))

Shape of X is : (50, 4)
Shape of y is : (50,)


In [47]:
#encode the categorical data using label and onehot encoder from sklearn
#importing library
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [48]:
labelencoder_X=LabelEncoder()
X[:,3]=labelencoder_X.fit_transform(X[:,3])
print("After encoding 'State' feature columns looks like:")
print(X[:,3])

After encoding 'State' feature columns looks like:
[2 0 1 2 1 2 0 1 2 0 1 0 1 0 1 2 0 2 1 2 0 2 1 1 2 0 1 2 1 2 1 2 0 1 0 2 1
 0 2 0 0 1 0 2 0 2 1 0 2 0]


In [49]:
#only label encoder is not enough because there might be bias in the model. so use onehotencoder to convert them to dummy variables
oneHotEncoder=OneHotEncoder(categorical_features=[3])
X=oneHotEncoder.fit_transform(X).toarray()

In [50]:
print(X.shape)

(50, 6)


In [53]:
#Make X such that there will be no dummy variable trap
#remove first column in X
X=X[:,1:]

In [54]:
print(X.shape)

(50, 5)


In [58]:
#split the dataset into training and testing sets
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [61]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(40, 5)
(40,)
(10, 5)
(10,)


In [63]:
#fitting multiple linear regression to training set
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [70]:
#predicting the test results
y_pred=regressor.predict(X_test)
print("Predicted values    Test values:")
for y1,y2 in zip(y_pred,y_test):
    print(str(y1)+"     "+str(y2))

Predicted values    Test values:
103015.20159796352     103282.38
132582.2776081537     144259.4
132447.73845174702     146121.95
71976.09851258088     77798.83
178537.48221055616     191050.39
116161.24230165542     105008.31
67851.69209675616     81229.06
98791.73374687255     97483.56
113969.43533012968     110352.25
167921.06569550867     166187.94


In [71]:
print("Error percentages in each prediction:")
for y1,y2 in zip(y_pred,y_test):
    print(abs(y1-y2)/y2 *100)

Error percentages in each prediction:
0.2586873017803101
8.094531373238967
9.358081758594784
7.484343257371765
6.54953271199491
10.620999711028038
16.468697167299286
1.3419429356832555
3.2778537185509853
1.0428709180152733


We need to start asking questions that is this the optimal model we can generate? We have used all the independent variables present in dataset. What if we have some highly statistical variables i.e variables which have high significance and variables which are not statistically significant? We need to remove these non statistically significant variables to make better predictions. We use Backward elimination and judge features based on P value
Steps in Backward elimination:
1. Select a significance level to stay in the model (Eg.SL= 0.05)
2. Fit the model with all possible predictors
3. Consider the predictor with highest P value. If P> SL, continue to next step else stop
4. Remove the predictor
5. Fit the model without this variable

In [72]:
#Building an optimal model using backward elimination
import statsmodels.formula.api as sm

In [81]:
#Add columns of 1s to generate bias term as statsmodels doesn't include this feature or intercept
X_new=np.append(np.ones((50,1)).astype(int),X,axis=1)

In [78]:
print(X.shape)
print(X_new.shape)
print(X_new[0])

(50, 5)
(50, 6)
[1.000000e+00 0.000000e+00 1.000000e+00 1.653492e+05 1.368978e+05
 4.717841e+05]


In [79]:
#Initialise X_opt (optimal features of X_new) as X_new (initially)
X_opt=X_new[:,[0,1,2,3,4,5]]

In [80]:
X_opt.shape

(50, 6)

In [83]:
# 2.Fit the model with all possible predictors
regressor_OLS=sm.OLS(y,X_opt).fit()

In [84]:
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Tue, 13 Nov 2018   Prob (F-statistic):           1.34e-27
Time:                        23:24:44   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.000    3.62e+04     6.4e+04
x1           198.7888   3371.007      0.059      0.953   -6595.030    6992.607
x2           -41.8870   3256.039     -0.013      0.990   -6604.003    6520.229
x3             0.8060      0.046     17.369      0.000       0.712       0.900
x4            -0.0270      0.052     -0.517      0.608      -0.132       0.078
x5             0.0270      0.017      1.574      0.123      -0.008       0.062
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [85]:
#if P value is above 5%, we need to remove the feature.
#as the feature with index 2 is about 95%, lets remove manually and repeat the same steps
X_opt=X_new[:,[0,1,3,4,5]]
regressor_OLS=sm.OLS(y,X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Tue, 13 Nov 2018   Prob (F-statistic):           8.49e-29
Time:                        23:37:40   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.011e+04   6647.870      7.537      0.000    3.67e+04    6.35e+04
x1           220.1585   2900.536      0.076      0.940   -5621.821    6062.138
x2             0.8060      0.046     17.606      0.000       0.714       0.898
x3            -0.0270      0.052     -0.523      0.604      -0.131       0.077
x4             0.0270      0.017      1.592      0.118      -0.007       0.061
==============================================================================
Omnibus:                       14.758   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.172
Skew:                          -0.948   Prob(JB):                     2.53e-05
Kurtosis:                       5.563   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [86]:
#now remove index1 as it have high P value
X_opt=X_new[:,[0,3,4,5]]
regressor_OLS=sm.OLS(y,X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Tue, 13 Nov 2018   Prob (F-statistic):           4.53e-30
Time:                        23:39:44   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.012e+04   6572.353      7.626      0.000    3.69e+04    6.34e+04
x1             0.8057      0.045     17.846      0.000       0.715       0.897
x2            -0.0268      0.051     -0.526      0.602      -0.130       0.076
x3             0.0272      0.016      1.655      0.105      -0.006       0.060
==============================================================================
Omnibus:                       14.838   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.442
Skew:                          -0.949   Prob(JB):                     2.21e-05
Kurtosis:                       5.586   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [87]:
#remove index 4 
#now remove index1 as it have high P value
X_opt=X_new[:,[0,3,5]]
regressor_OLS=sm.OLS(y,X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Tue, 13 Nov 2018   Prob (F-statistic):           2.16e-31
Time:                        23:40:59   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.000    4.16e+04    5.24e+04
x1             0.7966      0.041     19.266      0.000       0.713       0.880
x2             0.0299      0.016      1.927      0.060      -0.001       0.061
==============================================================================
Omnibus:                       14.677   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.161
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.575   Cond. No.                     5.32e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [88]:
#remove index 5 as still its greater than 5%
X_opt=X_new[:,[0,3]]
regressor_OLS=sm.OLS(y,X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Tue, 13 Nov 2018   Prob (F-statistic):           3.50e-32
Time:                        23:43:49   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.903e+04   2537.897     19.320      0.000    4.39e+04    5.41e+04
x1             0.8543      0.029     29.151      0.000       0.795       0.913
==============================================================================
Omnibus:                       13.727   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.536
Skew:                          -0.911   Prob(JB):                     9.44e-05
Kurtosis:                       5.361   Cond. No.                     1.65e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Conclusion
From the above backward elimination we can infer that Out of all independent variables only one independent variable(which is R&D Spend) is statistically significant